# Environment Settings

In [6]:
import torch
print(torch.version.cuda)


12.4


In [ ]:
import torch

if torch.cuda.is_available():
    gpu_id = torch.cuda.current_device()
    gpu_name = torch.cuda.get_device_name(gpu_id)
    gpu_capability = torch.cuda.get_device_capability(gpu_id)
    total_memory = torch.cuda.get_device_properties(gpu_id).total_memory
    
    print(f"Total number of GPU: {torch.cuda.device_count()}")  
    print(f"Total GPU memory: {total_memory / 1e9} GB")
    print(f"GPU ID: {gpu_id}")
    print(f"GPU Name: {gpu_name}")
    print(f"GPU Compute Capability: {gpu_capability}")
else:
    print("No GPU is available.")

Total number of GPU: 4
Total GPU memory: 51.033931776 GB
GPU ID: 2
GPU Name: NVIDIA RTX A6000
GPU Compute Capability: (8, 6)


In [ ]:
import torch


torch.cuda.set_device(2)


print("Current Device:", torch.cuda.current_device())
print("Device Name:", torch.cuda.get_device_name(torch.cuda.current_device()))
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device


Current Device: 2
Device Name: NVIDIA RTX A6000


device(type='cuda')

In [9]:
import warnings
warnings.filterwarnings('ignore')

# Load Data

In [10]:
import pandas as pd
import numpy as np

In [11]:
data = pd.read_csv('../dataset/national_illness.csv', index_col='date', parse_dates=True)

In [12]:
data.head()

,% WEIGHTED ILI,%UNWEIGHTED ILI,AGE 0-4,AGE 5-24,ILITOTAL,NUM. OF PROVIDERS,OT
date,,,,,,,
2002-01-01,1.22262,1.16668,582,805,2060,754,176569
2002-01-08,1.33344,1.21650,683,872,2267,785,186355
2002-01-15,1.31929,1.13057,642,878,2176,831,192469
2002-01-22,1.49484,1.25246,728,1045,2599,863,207512
2002-01-29,1.47195,1.30237,823,1189,2907,909,223208


In [13]:
print(data.dtypes)
print('shape: ', data.shape)

% WEIGHTED ILI       float64
%UNWEIGHTED ILI      float64
AGE 0-4                int64
AGE 5-24               int64
ILITOTAL               int64
NUM. OF PROVIDERS      int64
OT                     int64
dtype: object
shape:  (966, 7)


In [14]:
target = data['OT']
features = data.drop(['OT'],axis=1)

In [15]:
features['DATE'] = features.index.strftime('%Y%m%d%H').astype(int)

In [16]:
target.index

DatetimeIndex(['2002-01-01', '2002-01-08', '2002-01-15', '2002-01-22',
               '2002-01-29', '2002-02-05', '2002-02-12', '2002-02-19',
               '2002-02-26', '2002-03-05',
               ...
               '2020-04-28', '2020-05-05', '2020-05-12', '2020-05-19',
               '2020-05-26', '2020-06-02', '2020-06-09', '2020-06-16',
               '2020-06-23', '2020-06-30'],
              dtype='datetime64[ns]', name='date', length=966, freq=None)

In [ ]:
import torch
import numpy as np
import random

def set_seed(seed=42):
    random.seed(seed)                           
    np.random.seed(seed)                        
    torch.manual_seed(seed)                     
    torch.cuda.manual_seed(seed)                
    torch.cuda.manual_seed_all(seed)            
    torch.backends.cudnn.deterministic = True   
    torch.backends.cudnn.benchmark = False      

# Model Modification for DLinear (Decomposition-wise Look-ahead Augmentation)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

class moving_avg(nn.Module):
    """
    Moving average block to highlight the trend of time series
    """
    def __init__(self, kernel_size, stride):
        super(moving_avg, self).__init__()
        self.kernel_size = kernel_size
        self.avg = nn.AvgPool1d(kernel_size=kernel_size, stride=stride, padding=0)

    def forward(self, x):
        
        front = x[:, 0:1, :].repeat(1, (self.kernel_size - 1) // 2, 1)
        end = x[:, -1:, :].repeat(1, (self.kernel_size - 1) // 2, 1)
        x = torch.cat([front, x, end], dim=1)
        x = self.avg(x.permute(0, 2, 1))
        x = x.permute(0, 2, 1)
        return x


class series_decomp(nn.Module):
    """
    Series decomposition block
    """
    def __init__(self, kernel_size):
        super(series_decomp, self).__init__()
        self.moving_avg = moving_avg(kernel_size, stride=1)

    def forward(self, x):
        moving_mean = self.moving_avg(x)
        res = x - moving_mean
        return res, moving_mean

class Model(nn.Module):
    """
    Decomposition-Linear
    """
    def __init__(self, configs):
        super(Model, self).__init__()
        self.seq_len = configs.seq_len
        self.pred_len = configs.pred_len
        self.individual = configs.individual
        self.channels = configs.enc_in
        self.attach_to_trend = configs.attach_to_trend
        self.attach_to_seasonal = configs.attach_to_seasonal
        

        
        kernel_size = 25
        self.decompsition = series_decomp(kernel_size)
        

        
        self.seq_len_trend = self.seq_len + (self.pred_len // 3) if self.attach_to_trend else self.seq_len
        self.seq_len_seasonal = self.seq_len + (self.pred_len // 3) if self.attach_to_seasonal else self.seq_len

        
        if self.individual:
            self.Linear_Seasonal = nn.ModuleList()
            self.Linear_Trend = nn.ModuleList()
            
            for i in range(self.channels):
                self.Linear_Seasonal.append(nn.Linear(self.seq_len,self.pred_len))
                self.Linear_Trend.append(nn.Linear(self.seq_len,self.pred_len))

                
                
                
        else:
            self.Linear_Seasonal = nn.Linear(self.seq_len,self.pred_len)
            self.Linear_Trend = nn.Linear(self.seq_len,self.pred_len)
            
            
            
            

    def forward(self, x, ground_truth=None):
        
        seasonal_init, trend_init = self.decompsition(x)
        seasonal_init, trend_init = seasonal_init.permute(0,2,1), trend_init.permute(0,2,1)
        if ground_truth is not None:
            kernel_size = 25
            self.decompsition = series_decomp(kernel_size)
            ground_truth_seasonal, ground_truth_trend = self.decompsition(ground_truth)

            
            if self.seq_len_seasonal > self.seq_len:
                ground_truth_seasonal=ground_truth_seasonal.permute(0, 2, 1)
                
                seasonal_init = torch.cat([seasonal_init, ground_truth_seasonal], dim=2)
            if self.seq_len_trend > self.seq_len:
                ground_truth_trend=ground_truth_trend.permute(0, 2, 1)
                
                trend_init = torch.cat([trend_init, ground_truth_trend], dim=2)
        
        
        if self.individual:
            seasonal_output = torch.zeros([seasonal_init.size(0),seasonal_init.size(1),self.pred_len],dtype=seasonal_init.dtype).to(seasonal_init.device)
            trend_output = torch.zeros([trend_init.size(0),trend_init.size(1),self.pred_len],dtype=trend_init.dtype).to(trend_init.device)
            for i in range(self.channels):
                seasonal_output[:,i,:] = self.Linear_Seasonal[i](seasonal_init[:,i,:])
                trend_output[:,i,:] = self.Linear_Trend[i](trend_init[:,i,:])
        else:
            seasonal_output = self.Linear_Seasonal(seasonal_init)
            trend_output = self.Linear_Trend(trend_init)

        x = seasonal_output + trend_output
        return x.permute(0,2,1) 

# Split the Data

In [ ]:
import pandas as pd
import torch
import numpy as np
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, TensorDataset


target = data['OT'].values.reshape(-1, 1)  


seq_len  = 104
pred_len = 36


total_len  = len(target)
num_train  = int(total_len * 0.7)
num_test   = int(total_len * 0.2)
num_val    = total_len - num_train - num_test

border1s = [
    0,
    num_train - seq_len,
    total_len - num_test - seq_len
]
border2s = [
    num_train,
    num_train + num_val,
    total_len
]


train = target[border1s[0]:border2s[0]]
val   = target[border1s[1]:border2s[1]]
test  = target[border1s[2]:border2s[2]]


scaler = StandardScaler()
train = scaler.fit_transform(train)
val   = scaler.transform(val)
test  = scaler.transform(test)


def create_inout_sequences_univariate(data, seq_len, pred_len):
    seqs = []
    for i in range(len(data) - seq_len - pred_len + 1):
        seq_x = data[i : i + seq_len]
        seq_y = data[i + seq_len : i + seq_len + pred_len]
        seqs.append((seq_x, seq_y))
    return seqs

train_data = create_inout_sequences_univariate(train, seq_len, pred_len)
val_data   = create_inout_sequences_univariate(val,   seq_len, pred_len)
test_data  = create_inout_sequences_univariate(test,  seq_len, pred_len)


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_sequences = torch.tensor([x[0] for x in train_data]).float().to(device)
train_labels    = torch.tensor([x[1] for x in train_data]).float().to(device)

val_sequences   = torch.tensor([x[0] for x in val_data]).float().to(device)
val_labels      = torch.tensor([x[1] for x in val_data]).float().to(device)

test_sequences  = torch.tensor([x[0] for x in test_data]).float().to(device)
test_labels     = torch.tensor([x[1] for x in test_data]).float().to(device)


print("Train Sequences:", train_sequences.shape)
print("Train Labels:   ", train_labels.shape)
print("Val Sequences:  ", val_sequences.shape)
print("Val Labels:     ", val_labels.shape)
print("Test Sequences: ", test_sequences.shape)
print("Test Labels:    ", test_labels.shape)


Train Sequences: torch.Size([537, 104, 1])
Train Labels:    torch.Size([537, 36, 1])
Val Sequences:   torch.Size([62, 104, 1])
Val Labels:      torch.Size([62, 36, 1])
Test Sequences:  torch.Size([158, 104, 1])
Test Labels:     torch.Size([158, 36, 1])


In [20]:
set_seed(42)

In [ ]:

batch_size = 32


train_dataset = TensorDataset(train_sequences, train_labels)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)


val_dataset = TensorDataset(val_sequences, val_labels)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)


test_dataset = TensorDataset(test_sequences, test_labels)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


# 1st Model Training (Early Stopping)

In [22]:
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error


In [ ]:
import torch.optim as optim
import os


seq_len = 104
pred_len = 36  

class Configs:
    def __init__(self, seq_len, pred_len, enc_in, individual, learning_rate, lradj, patience, save_path, attach_to_trend, attach_to_seasonal):
        self.seq_len = seq_len
        self.pred_len = pred_len  
        self.enc_in = enc_in
        self.individual = individual
        self.learning_rate = learning_rate
        self.lradj = lradj
        self.patience = patience
        self.save_path = save_path
        self.attach_to_trend = attach_to_trend
        self.attach_to_seasonal = attach_to_seasonal

configs = Configs(seq_len=seq_len, pred_len=pred_len, enc_in=1, individual=False,
                   learning_rate=0.01, lradj='type1',  patience=3, save_path="./model_ILI", attach_to_trend=False , attach_to_seasonal=False) 


model = Model(configs).to(device)
loss_function = nn.MSELoss()  
optimizer = optim.Adam(model.parameters(), lr=configs.learning_rate)

if not os.path.exists(configs.save_path):
    os.makedirs(configs.save_path)


class EarlyStopping:
    def __init__(self, patience=3, verbose=False, delta=0):
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.inf
        self.delta = delta

    def __call__(self, val_loss, model, path):
        score = -val_loss
        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model, path)
        elif score < self.best_score + self.delta:
            self.counter += 1
            print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model, path)
            self.counter = 0

    def save_checkpoint(self, val_loss, model, path):
        if self.verbose:
            print(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        torch.save(model.state_dict(), os.path.join(path, 'checkpoint_ES.pth'))
        self.val_loss_min = val_loss




def adjust_learning_rate(optimizer, epoch, args):
    if args.lradj == 'type1':
        lr_adjust = {epoch: args.learning_rate * (0.5 ** ((epoch - 1) // 1))}
    elif args.lradj == 'type2':
        lr_adjust = {
            2: 5e-5, 4: 1e-5, 6: 5e-6, 8: 1e-6,
            10: 5e-7, 15: 1e-7, 20: 5e-8
        }
    elif args.lradj == '3':
        lr_adjust = {epoch: args.learning_rate if epoch < 10 else args.learning_rate*0.1}
    elif args.lradj == '4':
        lr_adjust = {epoch: args.learning_rate if epoch < 15 else args.learning_rate*0.1}
    elif args.lradj == '5':
        lr_adjust = {epoch: args.learning_rate if epoch < 25 else args.learning_rate*0.1}
    elif args.lradj == '6':
        lr_adjust = {epoch: args.learning_rate if epoch < 5 else args.learning_rate*0.1}  

    if epoch in lr_adjust.keys():
        lr = lr_adjust[epoch]
        for param_group in optimizer.param_groups:
            param_group['lr'] = lr
        print(f'Updating learning rate to {lr}')



def train_model(model, dataloader, val_dataloader, optimizer, loss_function, epochs, pred_len, args):
    early_stopping = EarlyStopping(patience=args.patience, verbose=True)
    for epoch in range(epochs):
        total_loss = 0
        model.train()
        
        
        adjust_learning_rate(optimizer, epoch, args)
        
        for batch_sequences, batch_labels in dataloader:
            optimizer.zero_grad()

            
            output = model(batch_sequences)
            
            
            
            output = output[:, -pred_len:, :]  
            batch_labels = batch_labels[:, -pred_len:, :]  
            
            
            assert output.shape == batch_labels.shape, f"Output shape {output.shape} and batch_labels shape {batch_labels.shape} must match"
            
            loss = loss_function(output, batch_labels)  
            
            loss.backward()  
            optimizer.step()  

            total_loss += loss.item()

        
        val_loss = 0
        model.eval()
        with torch.no_grad():
            for val_batch_sequences, val_batch_labels in val_dataloader:
                val_output = model(val_batch_sequences)
                val_output = val_output[:, -args.pred_len:, :]
                val_batch_labels = val_batch_labels[:, -args.pred_len:, :]
                
                val_loss += loss_function(val_output, val_batch_labels).item()
        
        val_loss /= len(val_dataloader)

        
        print(f'Epoch {epoch} | Train Loss: {total_loss / len(dataloader)} | Val Loss: {val_loss}')
        
        
        early_stopping(val_loss, model, args.save_path)
        if early_stopping.early_stop:
            print("Early stopping triggered. Stopping training.")
            break


train_model(model, train_dataloader, val_dataloader, optimizer, loss_function, epochs=10, pred_len=configs.pred_len, args=configs)




Updating learning rate to 0.02
Epoch 0 | Train Loss: 0.43952498628812675 | Val Loss: 0.17923186719417572
Validation loss decreased (inf --> 0.179232).  Saving model ...
Updating learning rate to 0.01
Epoch 1 | Train Loss: 0.12025273109183592 | Val Loss: 0.056933747604489326
Validation loss decreased (0.179232 --> 0.056934).  Saving model ...
Updating learning rate to 0.005
Epoch 2 | Train Loss: 0.07966581206111346 | Val Loss: 0.03994583897292614
Validation loss decreased (0.056934 --> 0.039946).  Saving model ...
Updating learning rate to 0.0025
Epoch 3 | Train Loss: 0.06965954689418569 | Val Loss: 0.03781148884445429
Validation loss decreased (0.039946 --> 0.037811).  Saving model ...
Updating learning rate to 0.00125
Epoch 4 | Train Loss: 0.06656054166309974 | Val Loss: 0.03437704499810934
Validation loss decreased (0.037811 --> 0.034377).  Saving model ...
Updating learning rate to 0.000625
Epoch 5 | Train Loss: 0.06518455572864588 | Val Loss: 0.03509111423045397
EarlyStopping count

# 1st Model Training (1 Epoch)

In [ ]:
import torch.optim as optim
import os



seq_len = 104
pred_len = 36  

class Configs:
    def __init__(self, seq_len, pred_len, enc_in, individual, learning_rate, lradj, patience, save_path, attach_to_trend, attach_to_seasonal):
        self.seq_len = seq_len
        self.pred_len = pred_len  
        self.enc_in = enc_in
        self.individual = individual
        self.learning_rate = learning_rate
        self.lradj = lradj
        self.patience = patience
        self.save_path = save_path
        self.attach_to_trend = attach_to_trend
        self.attach_to_seasonal = attach_to_seasonal

configs = Configs(seq_len=seq_len, pred_len=pred_len, enc_in=1, individual=False,
                   learning_rate=0.01, lradj='type1',  patience=3, save_path="./model_ILI", attach_to_trend=False , attach_to_seasonal=False) 


model = Model(configs).to(device)
loss_function = nn.MSELoss()  
optimizer = optim.Adam(model.parameters(), lr=configs.learning_rate)

if not os.path.exists(configs.save_path):
    os.makedirs(configs.save_path)


def adjust_learning_rate(optimizer, epoch, args):
    if args.lradj == 'type1':
        lr_adjust = {epoch: args.learning_rate * (0.5 ** ((epoch - 1) // 1))}
    elif args.lradj == 'type2':
        lr_adjust = {
            2: 5e-5, 4: 1e-5, 6: 5e-6, 8: 1e-6,
            10: 5e-7, 15: 1e-7, 20: 5e-8
        }
    elif args.lradj == '3':
        lr_adjust = {epoch: args.learning_rate if epoch < 10 else args.learning_rate*0.1}
    elif args.lradj == '4':
        lr_adjust = {epoch: args.learning_rate if epoch < 15 else args.learning_rate*0.1}
    elif args.lradj == '5':
        lr_adjust = {epoch: args.learning_rate if epoch < 25 else args.learning_rate*0.1}
    elif args.lradj == '6':
        lr_adjust = {epoch: args.learning_rate if epoch < 5 else args.learning_rate*0.1}  

    if epoch in lr_adjust.keys():
        lr = lr_adjust[epoch]
        for param_group in optimizer.param_groups:
            param_group['lr'] = lr
        print(f'Updating learning rate to {lr}')



def train_model(model, dataloader, val_dataloader, optimizer, loss_function, epochs, pred_len, args):
    
    for epoch in range(epochs):

        if epoch == 1:
            torch.save(model.state_dict(), os.path.join(args.save_path, 'checkpoint_1.pth'))
            print("Model saved at epoch 1")

        total_loss = 0
        model.train()
        
        
        adjust_learning_rate(optimizer, epoch, args)
        
        for batch_sequences, batch_labels in dataloader:
            optimizer.zero_grad()

            
            output = model(batch_sequences)
            
            
            
            output = output[:, -pred_len:, :]  
            batch_labels = batch_labels[:, -pred_len:, :]  
            
            
            assert output.shape == batch_labels.shape, f"Output shape {output.shape} and batch_labels shape {batch_labels.shape} must match"
            
            loss = loss_function(output, batch_labels)  
            
            loss.backward()  
            optimizer.step()  

            total_loss += loss.item()

        
        val_loss = 0
        model.eval()
        with torch.no_grad():
            for val_batch_sequences, val_batch_labels in val_dataloader:
                val_output = model(val_batch_sequences)
                val_output = val_output[:, -args.pred_len:, :]
                val_batch_labels = val_batch_labels[:, -args.pred_len:, :]
                
                val_loss += loss_function(val_output, val_batch_labels).item()
        
        val_loss /= len(val_dataloader)

        
        print(f'Epoch {epoch} | Train Loss: {total_loss / len(dataloader)} | Val Loss: {val_loss}')

    torch.save(model.state_dict(), os.path.join(args.save_path, f'checkpoint_{epochs}.pth'))
    print(f"Model saved at epoch {epochs}")
        
        
        
        
        
        


train_model(model, train_dataloader, val_dataloader, optimizer, loss_function, epochs=1, pred_len=configs.pred_len, args=configs)




Updating learning rate to 0.02
Epoch 0 | Train Loss: 0.47180766568464394 | Val Loss: 0.16696666181087494
Model saved at epoch 1


# Perform Inference and Form Predicted Value (PV) Sequences

In [ ]:
import torch.optim as optim
import os
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import torch


checkpoint_epochs = ['ES', 1]


test_1st_results = {}

for ckpt_epoch in checkpoint_epochs:
    print(f'\n[Predicting with model from epoch {ckpt_epoch}]')

    
    model_path = os.path.join(configs.save_path, f'checkpoint_{ckpt_epoch}.pth')
    
    model = Model(configs).to(device)
    
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.eval()

    predictions = []
    actuals = []

    with torch.no_grad():
        for batch_sequence, batch_label in test_dataloader:
            batch_sequence = batch_sequence.float().to(device)
            batch_label = batch_label.float().to(device)

            output = model(batch_sequence)
            output = output[:, -configs.pred_len:, :]
            batch_label = batch_label[:, -configs.pred_len:, :]

            predictions.append(output.detach().cpu().numpy())
            actuals.append(batch_label.detach().cpu().numpy())

    
    predictions = np.concatenate(predictions, axis=0)  
    actuals = np.concatenate(actuals, axis=0)

    
    predictions_flatten = predictions.flatten()
    actuals_flatten = actuals.flatten()

    
    mse = mean_squared_error(actuals_flatten, predictions_flatten)
    mae = mean_absolute_error(actuals_flatten, predictions_flatten)

    print(f'Epoch {ckpt_epoch} | MSE: {mse} | MAE: {mae}')

    
    num_samples = len(predictions_flatten) // configs.pred_len
    test_pv = torch.tensor(
        predictions_flatten.reshape(num_samples, configs.pred_len, 1),
        dtype=torch.float32
    )

    
    test_1st_results[ckpt_epoch] = {
        f'mse_test_E{ckpt_epoch}': mse,
        f'mae_test_E{ckpt_epoch}': mae,
        f'test_pv_E{ckpt_epoch}': test_pv
    }






[Predicting with model from epoch ES]
Epoch ES | MSE: 0.7942773699760437 | MAE: 0.7402907013893127

[Predicting with model from epoch 1]
Epoch 1 | MSE: 0.9989220499992371 | MAE: 0.7313221096992493


In [26]:
test_1st_results.keys()

dict_keys(['ES', 1])

In [ ]:
import torch.optim as optim
import os
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import torch


checkpoint_epochs = ['ES', 1]


val_1st_results = {}

for ckpt_epoch in checkpoint_epochs:
    print(f'\n[Predicting with model from epoch {ckpt_epoch}]')

    
    model_path = os.path.join(configs.save_path, f'checkpoint_{ckpt_epoch}.pth')
    
    model = Model(configs).to(device)
    
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.eval()

    predictions = []
    actuals = []

    with torch.no_grad():
        for batch_sequence, batch_label in val_dataloader:
            batch_sequence = batch_sequence.float().to(device)
            batch_label = batch_label.float().to(device)

            output = model(batch_sequence)
            output = output[:, -configs.pred_len:, :]
            batch_label = batch_label[:, -configs.pred_len:, :]

            predictions.append(output.detach().cpu().numpy())
            actuals.append(batch_label.detach().cpu().numpy())

    
    predictions = np.concatenate(predictions, axis=0)  
    actuals = np.concatenate(actuals, axis=0)

    
    predictions_flatten = predictions.flatten()
    actuals_flatten = actuals.flatten()

    
    mse = mean_squared_error(actuals_flatten, predictions_flatten)
    mae = mean_absolute_error(actuals_flatten, predictions_flatten)

    print(f'Epoch {ckpt_epoch} | MSE: {mse} | MAE: {mae}')

    
    num_samples = len(predictions_flatten) // configs.pred_len
    val_pv = torch.tensor(
        predictions_flatten.reshape(num_samples, configs.pred_len, 1),
        dtype=torch.float32
    )

    
    val_1st_results[ckpt_epoch] = {
        f'mse_val_E{ckpt_epoch}': mse,
        f'mae_val_E{ckpt_epoch}': mae,
        f'val_pv_E{ckpt_epoch}': val_pv
    }




[Predicting with model from epoch ES]
Epoch ES | MSE: 0.03416220098733902 | MAE: 0.14327122271060944

[Predicting with model from epoch 1]
Epoch 1 | MSE: 0.16682127118110657 | MAE: 0.33062130212783813


In [ ]:
import torch.optim as optim
import os
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import torch


train_dataloader_infer = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)


checkpoint_epochs = ['ES', 1]


train_1st_results = {}

for ckpt_epoch in checkpoint_epochs:
    print(f'\n[Predicting with model from epoch {ckpt_epoch}]')

    
    model_path = os.path.join(configs.save_path, f'checkpoint_{ckpt_epoch}.pth')
    
    model = Model(configs).to(device)
    
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.eval()

    predictions = []
    actuals = []

    with torch.no_grad():
        for batch_sequence, batch_label in train_dataloader_infer:
            batch_sequence = batch_sequence.float().to(device)
            batch_label = batch_label.float().to(device)

            output = model(batch_sequence)
            output = output[:, -configs.pred_len:, :]
            batch_label = batch_label[:, -configs.pred_len:, :]

            predictions.append(output.detach().cpu().numpy())
            actuals.append(batch_label.detach().cpu().numpy())

    
    predictions = np.concatenate(predictions, axis=0)  
    actuals = np.concatenate(actuals, axis=0)

    
    predictions_flatten = predictions.flatten()
    actuals_flatten = actuals.flatten()

    
    mse = mean_squared_error(actuals_flatten, predictions_flatten)
    mae = mean_absolute_error(actuals_flatten, predictions_flatten)

    print(f'Epoch {ckpt_epoch} | MSE: {mse} | MAE: {mae}')

    
    num_samples = len(predictions_flatten) // configs.pred_len
    train_pv = torch.tensor(
        predictions_flatten.reshape(num_samples, configs.pred_len, 1),
        dtype=torch.float32
    )

    
    train_1st_results[ckpt_epoch] = {
        f'mse_train_E{ckpt_epoch}': mse,
        f'mae_train_E{ckpt_epoch}': mae,
        f'train_pv_E{ckpt_epoch}': train_pv
    }




[Predicting with model from epoch ES]
Epoch ES | MSE: 0.0653376579284668 | MAE: 0.19289541244506836

[Predicting with model from epoch 1]
Epoch 1 | MSE: 0.18676164746284485 | MAE: 0.3474539518356323


In [29]:
print("train_sequences shape: ",train_sequences.shape)
print("val_sequences shape: ",val_sequences.shape)
print("test_sequences shape: ",test_sequences.shape)

print('---------------------------------------------')

print("train_pv shape: ",train_pv.shape)
print("val_pv shape: ",val_pv.shape)
print("test_pv shape: ",test_pv.shape)


print('---------------------------------------------')

print("train_labels shape: ",train_labels.shape)
print("val_labels shape: ",val_labels.shape)
print("test_labels shape: ",test_labels.shape)


train_sequences shape:  torch.Size([537, 104, 1])
val_sequences shape:  torch.Size([62, 104, 1])
test_sequences shape:  torch.Size([158, 104, 1])
---------------------------------------------
train_pv shape:  torch.Size([537, 36, 1])
val_pv shape:  torch.Size([62, 36, 1])
test_pv shape:  torch.Size([158, 36, 1])
---------------------------------------------
train_labels shape:  torch.Size([537, 36, 1])
val_labels shape:  torch.Size([62, 36, 1])
test_labels shape:  torch.Size([158, 36, 1])


# Segment Generation Function (sliding window-based)

In [ ]:
def create_sliding_segments(predictions, labels, segment_length, stride=1):
    """
    시퀀스에 슬라이딩 윈도우를 stride 간격으로 적용하여
    (B, num_segments, segment_length, C) 형태로 반환합니다.

    Args:
        predictions: Tensor of shape (B, T, C)
        labels: Tensor of shape (B, T, C)
        segment_length: int, 각 세그먼트의 길이 (예: pred_len // 3)
        stride: int, 슬라이딩 윈도우 stride 간격

    Returns:
        pred_segments: (B, num_segments, segment_length, C)
        label_segments: (B, num_segments, segment_length, C)
    """
    B, T, C = predictions.shape
    num_segments = (T - segment_length) // stride + 1

    pred_segments = []
    label_segments = []

    for i in range(0, T - segment_length + 1, stride):
        pred_seg = predictions[:, i:i+segment_length, :]  
        label_seg = labels[:, i:i+segment_length, :]      
        pred_segments.append(pred_seg.unsqueeze(1))       
        label_segments.append(label_seg.unsqueeze(1))

    pred_segments = torch.cat(pred_segments, dim=1)       
    label_segments = torch.cat(label_segments, dim=1)     

    return pred_segments, label_segments

# Segmentation for PV Sequence

In [ ]:
configs.pred_len 

36

In [ ]:
segment_len = configs.pred_len // 3
stride = 1  

train_segments_by_epoch = {}
val_segments_by_epoch = {}
test_segments_by_epoch = {}

for epoch in checkpoint_epochs:
    train_pv = train_1st_results[epoch][f'train_pv_E{epoch}']  
    val_pv = val_1st_results[epoch][f'val_pv_E{epoch}']        
    test_pv = test_1st_results[epoch][f'test_pv_E{epoch}']     

    
    train_pred_segments, train_label_segments = create_sliding_segments(train_pv, train_labels, segment_len,stride)
    val_pred_segments, val_label_segments = create_sliding_segments(val_pv, val_labels, segment_len, stride)
    test_pred_segments, test_label_segments = create_sliding_segments(test_pv, test_labels, segment_len, stride)

    train_segments_by_epoch[epoch] = {
        f'train_pred_segments_E{epoch}': train_pred_segments,  
        f'train_label_segments_E{epoch}': train_label_segments
    }

    val_segments_by_epoch[epoch] = {
        f'val_pred_segments_E{epoch}': val_pred_segments,
        f'val_label_segments_E{epoch}': val_label_segments
    }

    test_segments_by_epoch[epoch] = {
        f'test_pred_segments_E{epoch}': test_pred_segments,
        f'test_label_segments_E{epoch}': test_label_segments
    }

    
    print(f"[Epoch {epoch}] Train seg shape: {train_pred_segments.shape}, Val: {val_pred_segments.shape}, Test: {test_pred_segments.shape}")


[Epoch ES] Train seg shape: torch.Size([537, 25, 12, 1]), Val: torch.Size([62, 25, 12, 1]), Test: torch.Size([158, 25, 12, 1])
[Epoch 1] Train seg shape: torch.Size([537, 25, 12, 1]), Val: torch.Size([62, 25, 12, 1]), Test: torch.Size([158, 25, 12, 1])


## Save 1st Performance Metrics, PV, and Segment

In [ ]:
save_dir = './1st_results_ILI'  
os.makedirs(save_dir, exist_ok=True)


torch.save(train_1st_results, os.path.join(save_dir, 'train_1st_results.pt'))
torch.save(val_1st_results, os.path.join(save_dir, 'val_1st_results.pt'))
torch.save(test_1st_results, os.path.join(save_dir, 'test_1st_results.pt'))


torch.save(train_segments_by_epoch, os.path.join(save_dir, 'train_segments_by_epoch.pt'))
torch.save(val_segments_by_epoch, os.path.join(save_dir, 'val_segments_by_epoch.pt'))
torch.save(test_segments_by_epoch, os.path.join(save_dir, 'test_segments_by_epoch.pt'))

print("✅ All dictionaries saved successfully in ./saved_results")


✅ All dictionaries saved successfully in ./saved_results


# 2nd Model Training

In [34]:
train_segments_by_epoch = torch.load('./1st_results_ILI/train_segments_by_epoch.pt', weights_only=False)
val_segments_by_epoch = torch.load('./1st_results_ILI/val_segments_by_epoch.pt', weights_only=False)
test_segments_by_epoch = torch.load('./1st_results_ILI/test_segments_by_epoch.pt', weights_only=False)

In [35]:

print(train_segments_by_epoch.keys())
print(train_segments_by_epoch['ES'].keys())
print(train_segments_by_epoch[1].keys())

dict_keys(['ES', 1])
dict_keys(['train_pred_segments_EES', 'train_label_segments_EES'])
dict_keys(['train_pred_segments_E1', 'train_label_segments_E1'])


In [ ]:

train_pred_segments_ES = train_segments_by_epoch['ES']['train_pred_segments_EES']  
train_label_segments_ES = train_segments_by_epoch['ES']['train_label_segments_EES']
val_pred_segments_ES = val_segments_by_epoch['ES']['val_pred_segments_EES']  
val_label_segments_ES = val_segments_by_epoch['ES']['val_label_segments_EES']
test_pred_segments_ES = test_segments_by_epoch['ES']['test_pred_segments_EES']  
test_label_segments_ES = test_segments_by_epoch['ES']['test_label_segments_EES']


train_pred_segments_E1 = train_segments_by_epoch[1]['train_pred_segments_E1']  
train_label_segments_E1 = train_segments_by_epoch[1]['train_label_segments_E1']
val_pred_segments_E1 = val_segments_by_epoch[1]['val_pred_segments_E1']  
val_label_segments_E1 = val_segments_by_epoch[1]['val_label_segments_E1']
test_pred_segments_E1 = test_segments_by_epoch[1]['test_pred_segments_E1']  
test_label_segments_E1 = test_segments_by_epoch[1]['test_label_segments_E1']



## ES->1E

In [ ]:
import torch, torch.nn as nn, torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np, os
from pathlib import Path
import time


epo = 'ES'

class Configs:
    def __init__(self, seq_len, pred_len, enc_in, individual,
                 learning_rate, lradj, patience,
                 save_path, attach_to_trend, attach_to_seasonal):
        self.seq_len  = seq_len
        self.pred_len = pred_len
        self.enc_in   = enc_in          
        self.individual = individual
        self.learning_rate = learning_rate
        self.lradj = lradj
        self.patience = patience
        self.save_path = save_path
        self.attach_to_trend    = attach_to_trend
        self.attach_to_seasonal = attach_to_seasonal



base_cfg = Configs(
    seq_len          = 104 + pred_len//3,
    pred_len         = 36,
    enc_in           = 1,
    individual       = True,
    learning_rate    = 0.01,
    lradj            = 'type1',
    patience         = 3,
    save_path        = f"./model2_ILI_E{epo}_1E",
    attach_to_trend  = True,
    attach_to_seasonal = True
)
Path(base_cfg.save_path).mkdir(parents=True, exist_ok=True)

loss_fn = nn.MSELoss()


for seg in range(len(train_pred_segments_ES[0])):
    print(f"\n=========== Segment {seg} ===========")
    start_time = time.time()

    
    torch.cuda.reset_peak_memory_stats(device)
    mem_start = torch.cuda.memory_allocated(device)

    
    tr_pv = train_pred_segments_ES[:, seg, :, :]   
    va_pv = val_pred_segments_ES[:,  seg, :, :]

    tr_ds = TensorDataset(train_sequences, train_labels, tr_pv)
    va_ds = TensorDataset(val_sequences,   val_labels,   va_pv)
    tr_dl = DataLoader(tr_ds, batch_size=32, shuffle=True)
    va_dl = DataLoader(va_ds, batch_size=32, shuffle=False)

    
    model = Model(base_cfg).to(device)
    C     = model.channels                                    

    opts = []
    for c in range(C):
        params = list(model.Linear_Seasonal[c].parameters()) + \
                 list(model.Linear_Trend[c].parameters())
        opts.append(optim.Adam(params, lr=base_cfg.learning_rate))
        path = Path(base_cfg.save_path) / f"seg{seg}" / f"ch{c}"
        path.mkdir(parents=True, exist_ok=True)
        
    active = set(range(C))            

    
    for epoch in range(1, 2):
        if not active: break          

        
        model.train()
        for c in active:                              
            if base_cfg.lradj=='type1':
                lr = base_cfg.learning_rate * 0.5**((epoch-1)//1)
                for g in opts[c].param_groups: g['lr'] = lr

        printed_tensor_info = False  

        for xb, yb, pv in tr_dl:
            xb, yb, pv = xb.float().to(device), yb.float().to(device), pv.float().to(device)

            if not printed_tensor_info:
                
                xb_size = xb.numel() * xb.element_size() / 1024**2
                yb_size = yb.numel() * yb.element_size() / 1024**2
                pv_size = pv.numel() * pv.element_size() / 1024**2

                
                
                num_batches = len(tr_dl)

                
                total_xb_MB = num_batches * xb_size
                total_yb_MB = num_batches * yb_size
                total_pv_MB = num_batches * pv_size
                


                print(f"[Segment {seg}] Batch Size Info:")
                print(f"  xb per batch = {xb_size:.4f} MB, total = {total_xb_MB:.4f} MB")
                print(f"  yb per batch = {yb_size:.4f} MB, total = {total_yb_MB:.4f} MB")
                print(f"  pv per batch = {pv_size:.4f} MB, total = {total_pv_MB:.4f} MB")

                printed_tensor_info = True


            out = model(xb, pv)                       

            loss_list = []
            for c in active:
                pred = out[:, -base_cfg.pred_len:, c]
                tgt  = yb[:, -base_cfg.pred_len:, c]
                loss_list.append(loss_fn(pred, tgt))

            total = torch.stack(loss_list).sum()
            for opt in opts:
                if opt: opt.zero_grad()
            total.backward()
            for c in active:
                opts[c].step()
        
    elapsed = time.time() - start_time
    mem_end = torch.cuda.memory_allocated(device)
    peak_mem = torch.cuda.max_memory_allocated(device)


    print(f"[Segment {seg}] Training Time = {elapsed:.2f} sec")
    print(f"[Segment {seg}] GPU Memory Start = {mem_start/1024**2:.2f} MB")
    print(f"[Segment {seg}] GPU Memory End   = {mem_end/1024**2:.2f} MB")
    print(f"[Segment {seg}] GPU Peak Memory  = {peak_mem/1024**2:.2f} MB")

    print(f"\n============= Done =============")

    
    torch.save(model.state_dict(),
                Path(base_cfg.save_path) / f"seg{seg}" / f"ch{c}" / "checkpoint.pth")
    del model, opts; torch.cuda.empty_cache()






=========== Segment 0 ===========
[Segment 0] Batch Size Info:
  xb per batch = 0.0127 MB, total = 0.2158 MB
  yb per batch = 0.0044 MB, total = 0.0747 MB
  pv per batch = 0.0015 MB, total = 0.0249 MB
[Segment 0] Training Time = 0.10 sec
[Segment 0] GPU Memory Start = 19.43 MB
[Segment 0] GPU Memory End   = 19.55 MB
[Segment 0] GPU Peak Memory  = 20.61 MB

============= Done =============

=========== Segment 1 ===========
[Segment 1] Batch Size Info:
  xb per batch = 0.0127 MB, total = 0.2158 MB
  yb per batch = 0.0044 MB, total = 0.0747 MB
  pv per batch = 0.0015 MB, total = 0.0249 MB
[Segment 1] Training Time = 0.06 sec
[Segment 1] GPU Memory Start = 19.55 MB
[Segment 1] GPU Memory End   = 19.59 MB
[Segment 1] GPU Peak Memory  = 20.64 MB

============= Done =============

=========== Segment 2 ===========
[Segment 2] Batch Size Info:
  xb per batch = 0.0127 MB, total = 0.2158 MB
  yb per batch = 0.0044 MB, total = 0.0747 MB
  pv per batch = 0.0015 MB, total = 0.0249 MB
[Segment 2] 

## 1E->1E

In [ ]:
import torch, torch.nn as nn, torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np, os
from pathlib import Path


epo = 1


class Configs:
    def __init__(self, seq_len, pred_len, enc_in, individual,
                 learning_rate, lradj, patience,
                 save_path, attach_to_trend, attach_to_seasonal):
        self.seq_len  = seq_len
        self.pred_len = pred_len
        self.enc_in   = enc_in          
        self.individual = individual
        self.learning_rate = learning_rate
        self.lradj = lradj
        self.patience = patience
        self.save_path = save_path
        self.attach_to_trend    = attach_to_trend
        self.attach_to_seasonal = attach_to_seasonal



base_cfg = Configs(
    seq_len          = 104 + pred_len//3,
    pred_len         = 36,
    enc_in           = 1,
    individual       = True,
    learning_rate    = 0.01,
    lradj            = 'type1',
    patience         = 3,
    save_path        = f"./model2_ILI_E{epo}_1E/",
    attach_to_trend  = True,
    attach_to_seasonal = True
)
Path(base_cfg.save_path).mkdir(parents=True, exist_ok=True)

loss_fn = nn.MSELoss()



for seg in range(len(train_pred_segments_E1[0])):
    print(f"\n=========== Segment {seg} ===========")
    
    start_time = time.time()

    
    torch.cuda.reset_peak_memory_stats(device)
    mem_start = torch.cuda.memory_allocated(device)

    
    tr_pv = train_pred_segments_E1[:, seg, :, :]   
    va_pv = val_pred_segments_E1[:,  seg, :, :]

    tr_ds = TensorDataset(train_sequences, train_labels, tr_pv)
    va_ds = TensorDataset(val_sequences,   val_labels,   va_pv)
    tr_dl = DataLoader(tr_ds, batch_size=32, shuffle=True)
    va_dl = DataLoader(va_ds, batch_size=32, shuffle=False)

    
    model = Model(base_cfg).to(device)
    C     = model.channels                                    

    opts = []
    for c in range(C):
        params = list(model.Linear_Seasonal[c].parameters()) + \
                 list(model.Linear_Trend[c].parameters())
        opts.append(optim.Adam(params, lr=base_cfg.learning_rate))
        path = Path(base_cfg.save_path) / f"seg{seg}" / f"ch{c}"
        path.mkdir(parents=True, exist_ok=True)
        
    active = set(range(C))            

    
    for epoch in range(1, 2):
        if not active: break          

        
        model.train()
        for c in active:                              
            if base_cfg.lradj=='type1':
                lr = base_cfg.learning_rate * 0.5**((epoch-1)//1)
                for g in opts[c].param_groups: g['lr'] = lr

        printed_tensor_info = False
        
        for xb, yb, pv in tr_dl:
            xb, yb, pv = xb.float().to(device), yb.float().to(device), pv.float().to(device)

            if not printed_tensor_info:
                
                xb_size = xb.numel() * xb.element_size() / 1024**2
                yb_size = yb.numel() * yb.element_size() / 1024**2
                pv_size = pv.numel() * pv.element_size() / 1024**2

                
                
                num_batches = len(tr_dl)

                
                total_xb_MB = num_batches * xb_size
                total_yb_MB = num_batches * yb_size
                total_pv_MB = num_batches * pv_size
                


                print(f"[Segment {seg}] Batch Size Info:")
                print(f"  xb per batch = {xb_size:.4f} MB, total = {total_xb_MB:.4f} MB")
                print(f"  yb per batch = {yb_size:.4f} MB, total = {total_yb_MB:.4f} MB")
                print(f"  pv per batch = {pv_size:.4f} MB, total = {total_pv_MB:.4f} MB")

                printed_tensor_info = True



            out = model(xb, pv)                       

            loss_list = []
            for c in active:
                pred = out[:, -base_cfg.pred_len:, c]
                tgt  = yb[:, -base_cfg.pred_len:, c]
                loss_list.append(loss_fn(pred, tgt))

            total = torch.stack(loss_list).sum()
            for opt in opts:
                if opt: opt.zero_grad()
            total.backward()
            for c in active:
                opts[c].step()

        
    elapsed = time.time() - start_time
    mem_end = torch.cuda.memory_allocated(device)
    peak_mem = torch.cuda.max_memory_allocated(device)


    print(f"[Segment {seg}] Training Time = {elapsed:.2f} sec")
    print(f"[Segment {seg}] GPU Memory Start = {mem_start/1024**2:.2f} MB")
    print(f"[Segment {seg}] GPU Memory End   = {mem_end/1024**2:.2f} MB")
    print(f"[Segment {seg}] GPU Peak Memory  = {peak_mem/1024**2:.2f} MB")

    print(f"\n============= Done =============")

    
    torch.save(model.state_dict(),
                Path(base_cfg.save_path) / f"seg{seg}" / f"ch{c}" / "checkpoint.pth")
    del model, opts; torch.cuda.empty_cache()



=========== Segment 0 ===========
[Segment 0] Batch Size Info:
  xb per batch = 0.0127 MB, total = 0.2158 MB
  yb per batch = 0.0044 MB, total = 0.0747 MB
  pv per batch = 0.0015 MB, total = 0.0249 MB
[Segment 0] Training Time = 0.08 sec
[Segment 0] GPU Memory Start = 19.55 MB
[Segment 0] GPU Memory End   = 19.55 MB
[Segment 0] GPU Peak Memory  = 20.64 MB

============= Done =============

=========== Segment 1 ===========
[Segment 1] Batch Size Info:
  xb per batch = 0.0127 MB, total = 0.2158 MB
  yb per batch = 0.0044 MB, total = 0.0747 MB
  pv per batch = 0.0015 MB, total = 0.0249 MB
[Segment 1] Training Time = 0.06 sec
[Segment 1] GPU Memory Start = 19.55 MB
[Segment 1] GPU Memory End   = 19.59 MB
[Segment 1] GPU Peak Memory  = 20.64 MB

============= Done =============

=========== Segment 2 ===========
[Segment 2] Batch Size Info:
  xb per batch = 0.0127 MB, total = 0.2158 MB
  yb per batch = 0.0044 MB, total = 0.0747 MB
  pv per batch = 0.0015 MB, total = 0.0249 MB
[Segment 2] 

# 2nd Inference for Validation Set

In [39]:
train_segments_by_epoch = torch.load('./1st_results_ILI/train_segments_by_epoch.pt', weights_only=False)
val_segments_by_epoch = torch.load('./1st_results_ILI/val_segments_by_epoch.pt', weights_only=False)
test_segments_by_epoch = torch.load('./1st_results_ILI/test_segments_by_epoch.pt', weights_only=False)

In [ ]:
train_pred_segments_ES = train_segments_by_epoch['ES']['train_pred_segments_EES']  
train_label_segments_ES = train_segments_by_epoch['ES']['train_label_segments_EES']
val_pred_segments_ES = val_segments_by_epoch['ES']['val_pred_segments_EES']  
val_label_segments_ES = val_segments_by_epoch['ES']['val_label_segments_EES']
test_pred_segments_ES = test_segments_by_epoch['ES']['test_pred_segments_EES']  
test_label_segments_ES = test_segments_by_epoch['ES']['test_label_segments_EES']


train_pred_segments_E1 = train_segments_by_epoch[1]['train_pred_segments_E1']  
train_label_segments_E1 = train_segments_by_epoch[1]['train_label_segments_E1']
val_pred_segments_E1 = val_segments_by_epoch[1]['val_pred_segments_E1']  
val_label_segments_E1 = val_segments_by_epoch[1]['val_label_segments_E1']
test_pred_segments_E1 = test_segments_by_epoch[1]['test_pred_segments_E1']  
test_label_segments_E1 = test_segments_by_epoch[1]['test_label_segments_E1']


## ES->1E

### Configs

In [ ]:
import torch, torch.nn as nn, torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np, os
from pathlib import Path


class Configs:
    def __init__(self, seq_len, pred_len, enc_in, individual,
                 learning_rate, lradj, patience,
                 save_path, attach_to_trend, attach_to_seasonal):
        self.seq_len  = seq_len
        self.pred_len = pred_len
        self.enc_in   = enc_in          
        self.individual = individual
        self.learning_rate = learning_rate
        self.lradj = lradj
        self.patience = patience
        self.save_path = save_path
        self.attach_to_trend    = attach_to_trend
        self.attach_to_seasonal = attach_to_seasonal


base_cfg = Configs(
    pred_len         = 36,
    seq_len          = 104 + pred_len//3,
    enc_in           = 1,
    individual       = True,
    learning_rate    = 0.01,
    lradj            = 'type1',
    patience         = 3,
    save_path        = "./model2_ILI_EES_1E",
    attach_to_trend  = True,
    attach_to_seasonal = True
)
Path(base_cfg.save_path).mkdir(parents=True, exist_ok=True)

### Inference

In [ ]:
import torch, numpy as np
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import r2_score
from pathlib import Path


C        = 1                                
P        = base_cfg.pred_len                
seg_cnt  = 25
batch_sz = 32

results2_ILI_ES_val_1E = {}  

for seg in range(seg_cnt):
    print(f"\n―――― Inference  |  Segment {seg} ――――")

    
    pv = val_pred_segments_ES[:, seg, :, :]         
    ds = TensorDataset(val_sequences, val_labels, pv) 
    dl = DataLoader(ds, batch_size=batch_sz, shuffle=False)

    
    pred_ch, true_ch, metrics = [], [], {}

    
    for ch in range(C):
        ckpt = Path(base_cfg.save_path) / f"seg{seg}" / f"ch{ch}" / "checkpoint.pth"
        if not ckpt.exists():
            print(f"[Warn] seg{seg}-ch{ch} ckpt 없음 → skip");  continue

        
        model = Model(base_cfg).to(device)
        model.load_state_dict(torch.load(ckpt, map_location=device))
        model.eval()

        preds, trues = [], []
        with torch.no_grad():
            for xb, yb, pv_batch in dl:                 
                xb  = xb.float().to(device)
                yb  = yb.float()      
                pv_ = pv_batch.float().to(device)

                out = model(xb, pv_)                     
                preds.append(out[:, -P:, ch].cpu())
                trues.append(yb[:, -P:, ch])

        preds = torch.cat(preds, 0)     
        trues = torch.cat(trues, 0)
        pred_ch.append(preds)
        true_ch.append(trues)

        
        p, t = preds.reshape(-1).cpu().numpy(), trues.reshape(-1).cpu().numpy()
        mse  = np.mean((p - t)**2)
        mae  = np.mean(np.abs(p - t))
        metrics[ch] = (mse, mae)

        print(f"  seg{seg}-ch{ch}  MSE:{mse}  MAE:{mae}")

        del model; torch.cuda.empty_cache()

    
    pred_all = torch.stack(pred_ch,  dim=2)   
    true_all = torch.stack(true_ch,  dim=2)

    
    mse_avg = np.mean([m for m,_ in metrics.values()])
    mae_avg = np.mean([a for _,a in metrics.values()])
    print(f"▶ seg{seg}  mean  MSE:{mse_avg}  MAE:{mae_avg}")

    
    results2_ILI_ES_val_1E[seg] = {
        "pred2_ES_val_1E"    : pred_all,   
        "true2_ES_val_1E"    : true_all,   
        "metrics2_ES_val_1E" : metrics     
    }


seg10_ch0_mse = results2_ILI_ES_val_1E[10]["metrics2_ES_val_1E"][0][0]
print("\nseg10 ch0 MSE :", seg10_ch0_mse)



―――― Inference  |  Segment 0 ――――


  seg0-ch0  MSE:0.11843638122081757  MAE:0.2750007212162018
▶ seg0  mean  MSE:0.11843638122081757  MAE:0.2750007212162018

―――― Inference  |  Segment 1 ――――
  seg1-ch0  MSE:0.1267305314540863  MAE:0.28982484340667725
▶ seg1  mean  MSE:0.1267305314540863  MAE:0.28982484340667725

―――― Inference  |  Segment 2 ――――
  seg2-ch0  MSE:0.1339370310306549  MAE:0.296939492225647
▶ seg2  mean  MSE:0.1339370310306549  MAE:0.296939492225647

―――― Inference  |  Segment 3 ――――
  seg3-ch0  MSE:0.14535804092884064  MAE:0.30447688698768616
▶ seg3  mean  MSE:0.14535804092884064  MAE:0.30447688698768616

―――― Inference  |  Segment 4 ――――
  seg4-ch0  MSE:0.07706859707832336  MAE:0.21521764993667603
▶ seg4  mean  MSE:0.07706859707832336  MAE:0.21521764993667603

―――― Inference  |  Segment 5 ――――
  seg5-ch0  MSE:0.128976970911026  MAE:0.28885096311569214
▶ seg5  mean  MSE:0.128976970911026  MAE:0.28885096311569214

―――― Inference  |  Segment 6 ――――
  seg6-ch0  MSE:0.14387531578540802  MAE:0.30111461877822876

## 1E->1E

### Configs

In [ ]:
import torch, torch.nn as nn, torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np, os
from pathlib import Path


class Configs:
    def __init__(self, seq_len, pred_len, enc_in, individual,
                 learning_rate, lradj, patience,
                 save_path, attach_to_trend, attach_to_seasonal):
        self.seq_len  = seq_len
        self.pred_len = pred_len
        self.enc_in   = enc_in          
        self.individual = individual
        self.learning_rate = learning_rate
        self.lradj = lradj
        self.patience = patience
        self.save_path = save_path
        self.attach_to_trend    = attach_to_trend
        self.attach_to_seasonal = attach_to_seasonal


base_cfg = Configs(
    pred_len         = 36,
    seq_len          = 104 + pred_len//3,
    enc_in           = 1,
    individual       = True,
    learning_rate    = 0.01,
    lradj            = 'type1',
    patience         = 3,
    save_path        = "./model2_ILI_E1_1E",
    attach_to_trend  = True,
    attach_to_seasonal = True
)
Path(base_cfg.save_path).mkdir(parents=True, exist_ok=True)

### Inference

In [ ]:
import torch, numpy as np
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import r2_score
from pathlib import Path


C        = 1                                
P        = base_cfg.pred_len                
seg_cnt  = 25
batch_sz = 32

results2_ILI_E1_val_1E = {}  

for seg in range(seg_cnt):
    print(f"\n―――― Inference  |  Segment {seg} ――――")

    
    pv = val_pred_segments_E1[:, seg, :, :]         
    ds = TensorDataset(val_sequences, val_labels, pv) 
    dl = DataLoader(ds, batch_size=batch_sz, shuffle=False)

    
    pred_ch, true_ch, metrics = [], [], {}

    
    for ch in range(C):
        ckpt = Path(base_cfg.save_path) / f"seg{seg}" / f"ch{ch}" / "checkpoint.pth"
        if not ckpt.exists():
            print(f"[Warn] seg{seg}-ch{ch} ckpt 없음 → skip");  continue

        
        model = Model(base_cfg).to(device)
        model.load_state_dict(torch.load(ckpt, map_location=device))
        model.eval()

        preds, trues = [], []
        with torch.no_grad():
            for xb, yb, pv_batch in dl:                 
                xb  = xb.float().to(device)
                yb  = yb.float()        
                pv_ = pv_batch.float().to(device)

                out = model(xb, pv_)                     
                preds.append(out[:, -P:, ch].cpu())
                trues.append(yb[:, -P:, ch])

        preds = torch.cat(preds, 0)     
        trues = torch.cat(trues, 0)
        pred_ch.append(preds)
        true_ch.append(trues)

        
        p, t = preds.reshape(-1).cpu().numpy(), trues.reshape(-1).cpu().numpy()
        mse  = np.mean((p - t)**2)
        mae  = np.mean(np.abs(p - t))
        metrics[ch] = (mse, mae)

        print(f"  seg{seg}-ch{ch}  MSE:{mse}  MAE:{mae}")

        del model; torch.cuda.empty_cache()

    
    pred_all = torch.stack(pred_ch,  dim=2)   
    true_all = torch.stack(true_ch,  dim=2)

    
    mse_avg = np.mean([m for m,_ in metrics.values()])
    mae_avg = np.mean([a for _,a in metrics.values()])
    print(f"▶ seg{seg}  mean  MSE:{mse_avg}  MAE:{mae_avg}")

    
    results2_ILI_E1_val_1E[seg] = {
        "pred2_E1_val_1E"    : pred_all,   
        "true2_E1_val_1E"    : true_all,   
        "metrics2_E1_val_1E" : metrics     
    }


seg10_ch0_mse = results2_ILI_E1_val_1E[10]["metrics2_E1_val_1E"][0][0]
print("\nseg10 ch0 MSE :", seg10_ch0_mse)



―――― Inference  |  Segment 0 ――――
  seg0-ch0  MSE:0.08775671571493149  MAE:0.2362368404865265
▶ seg0  mean  MSE:0.08775671571493149  MAE:0.2362368404865265

―――― Inference  |  Segment 1 ――――
  seg1-ch0  MSE:0.13901188969612122  MAE:0.30100521445274353
▶ seg1  mean  MSE:0.13901188969612122  MAE:0.30100521445274353

―――― Inference  |  Segment 2 ――――
  seg2-ch0  MSE:0.14594338834285736  MAE:0.3037841320037842
▶ seg2  mean  MSE:0.14594338834285736  MAE:0.3037841320037842

―――― Inference  |  Segment 3 ――――
  seg3-ch0  MSE:0.11851231008768082  MAE:0.27237212657928467
▶ seg3  mean  MSE:0.11851231008768082  MAE:0.27237212657928467

―――― Inference  |  Segment 4 ――――
  seg4-ch0  MSE:0.1198359951376915  MAE:0.2693749666213989
▶ seg4  mean  MSE:0.1198359951376915  MAE:0.2693749666213989

―――― Inference  |  Segment 5 ――――
  seg5-ch0  MSE:0.12919865548610687  MAE:0.29205259680747986
▶ seg5  mean  MSE:0.12919865548610687  MAE:0.29205259680747986

―――― Inference  |  Segment 6 ――――
  seg6-ch0  MSE:0.1

  seg12-ch0  MSE:0.14044496417045593  MAE:0.3116888105869293
▶ seg12  mean  MSE:0.14044496417045593  MAE:0.3116888105869293

―――― Inference  |  Segment 13 ――――
  seg13-ch0  MSE:0.16904135048389435  MAE:0.3277185261249542
▶ seg13  mean  MSE:0.16904135048389435  MAE:0.3277185261249542

―――― Inference  |  Segment 14 ――――
  seg14-ch0  MSE:0.15661098062992096  MAE:0.3194475471973419
▶ seg14  mean  MSE:0.15661098062992096  MAE:0.3194475471973419

―――― Inference  |  Segment 15 ――――
  seg15-ch0  MSE:0.10423111915588379  MAE:0.2551555633544922
▶ seg15  mean  MSE:0.10423111915588379  MAE:0.2551555633544922

―――― Inference  |  Segment 16 ――――
  seg16-ch0  MSE:0.11217311769723892  MAE:0.2644719183444977
▶ seg16  mean  MSE:0.11217311769723892  MAE:0.2644719183444977

―――― Inference  |  Segment 17 ――――
  seg17-ch0  MSE:0.15973368287086487  MAE:0.32069385051727295
▶ seg17  mean  MSE:0.15973368287086487  MAE:0.32069385051727295

―――― Inference  |  Segment 18 ――――
  seg18-ch0  MSE:0.10384123027324677  

# 2nd Inference for Test Set

## ES->1E

### Configs

In [ ]:
import torch, torch.nn as nn, torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np, os
from pathlib import Path

pred_len =36
seq_len = 104 + pred_len//3


class Configs:
    def __init__(self, seq_len, pred_len, enc_in, individual,
                 learning_rate, lradj, patience,
                 save_path, attach_to_trend, attach_to_seasonal):
        self.seq_len  = seq_len
        self.pred_len = pred_len
        self.enc_in   = enc_in          
        self.individual = individual
        self.learning_rate = learning_rate
        self.lradj = lradj
        self.patience = patience
        self.save_path = save_path
        self.attach_to_trend    = attach_to_trend
        self.attach_to_seasonal = attach_to_seasonal


base_cfg = Configs(
    pred_len         = 36,
    seq_len          = 104 + pred_len//3,
    enc_in           = 1,
    individual       = True,
    learning_rate    = 0.01,
    lradj            = 'type1',
    patience         = 3,
    save_path        = "./model2_ILI_EES_1E",
    attach_to_trend  = True,
    attach_to_seasonal = True
)
Path(base_cfg.save_path).mkdir(parents=True, exist_ok=True)

### Inference

In [ ]:
import torch, numpy as np
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import r2_score
from pathlib import Path

C        = 1                                
P        = base_cfg.pred_len                
seg_cnt  = 25
batch_sz = 32

results2_ILI_ES_test_1E = {}  

for seg in range(seg_cnt):
    print(f"\n―――― Inference  |  Segment {seg} ――――")

    
    pv = test_pred_segments_ES[:, seg, :, :]         
    ds = TensorDataset(test_sequences, test_labels, pv) 
    dl = DataLoader(ds, batch_size=batch_sz, shuffle=False)

    
    pred_ch, true_ch, metrics = [], [], {}

    
    for ch in range(C):
        ckpt = Path(base_cfg.save_path) / f"seg{seg}" / f"ch{ch}" / "checkpoint.pth"
        if not ckpt.exists():
            print(f"[Warn] seg{seg}-ch{ch} ckpt 없음 → skip");  continue

        
        model = Model(base_cfg).to(device)
        model.load_state_dict(torch.load(ckpt, map_location=device))
        model.eval()

        preds, trues = [], []
        with torch.no_grad():
            for xb, yb, pv_batch in dl:                 
                xb  = xb.float().to(device)
                yb  = yb.float()        
                pv_ = pv_batch.float().to(device)

                out = model(xb, pv_)                     
                preds.append(out[:, -P:, ch].cpu())
                trues.append(yb[:, -P:, ch])

        preds = torch.cat(preds, 0)     
        trues = torch.cat(trues, 0)
        pred_ch.append(preds)
        true_ch.append(trues)

        
        p, t = preds.reshape(-1).cpu().numpy(), trues.reshape(-1).cpu().numpy()
        mse  = np.mean((p - t)**2)
        mae  = np.mean(np.abs(p - t))
        metrics[ch] = (mse, mae)

        print(f"  seg{seg}-ch{ch}  MSE:{mse}  MAE:{mae}")

        del model; torch.cuda.empty_cache()

    
    pred_all = torch.stack(pred_ch,  dim=2)   
    true_all = torch.stack(true_ch,  dim=2)

    
    mse_avg = np.mean([m for m,_ in metrics.values()])
    mae_avg = np.mean([a for _,a in metrics.values()])
    print(f"▶ seg{seg}  mean  MSE:{mse_avg}  MAE:{mae_avg}")

    
    results2_ILI_ES_test_1E[seg] = {
        "pred2_ES_test_1E"    : pred_all,   
        "true2_ES_test_1E"    : true_all,   
        "metrics2_ES_test_1E" : metrics     
    }


seg10_ch0_mse = results2_ILI_ES_test_1E[10]["metrics2_ES_test_1E"][0][0]
print("\nseg10 ch0 MSE :", seg10_ch0_mse)



―――― Inference  |  Segment 0 ――――
  seg0-ch0  MSE:0.8691173791885376  MAE:0.6881468296051025
▶ seg0  mean  MSE:0.8691173791885376  MAE:0.6881468296051025

―――― Inference  |  Segment 1 ――――
  seg1-ch0  MSE:1.16600501537323  MAE:0.8234830498695374
▶ seg1  mean  MSE:1.16600501537323  MAE:0.8234830498695374

―――― Inference  |  Segment 2 ――――
  seg2-ch0  MSE:0.9332036972045898  MAE:0.7293354272842407
▶ seg2  mean  MSE:0.9332036972045898  MAE:0.7293354272842407

―――― Inference  |  Segment 3 ――――
  seg3-ch0  MSE:1.008171796798706  MAE:0.7839934229850769
▶ seg3  mean  MSE:1.008171796798706  MAE:0.7839934229850769

―――― Inference  |  Segment 4 ――――
  seg4-ch0  MSE:0.8597521781921387  MAE:0.7130287885665894
▶ seg4  mean  MSE:0.8597521781921387  MAE:0.7130287885665894

―――― Inference  |  Segment 5 ――――
  seg5-ch0  MSE:1.0093908309936523  MAE:0.7667261958122253
▶ seg5  mean  MSE:1.0093908309936523  MAE:0.7667261958122253

―――― Inference  |  Segment 6 ――――
  seg6-ch0  MSE:0.8121375441551208  MAE:0

## 1E->1E

### Configs

In [ ]:
import torch, torch.nn as nn, torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np, os
from pathlib import Path

pred_len = 36
seq_len = 104 + pred_len//3


class Configs:
    def __init__(self, seq_len, pred_len, enc_in, individual,
                 learning_rate, lradj, patience,
                 save_path, attach_to_trend, attach_to_seasonal):
        self.seq_len  = seq_len
        self.pred_len = pred_len
        self.enc_in   = enc_in          
        self.individual = individual
        self.learning_rate = learning_rate
        self.lradj = lradj
        self.patience = patience
        self.save_path = save_path
        self.attach_to_trend    = attach_to_trend
        self.attach_to_seasonal = attach_to_seasonal


base_cfg = Configs(
    pred_len         = 36,
    seq_len          = 104 + pred_len//3,
    enc_in           = 1,
    individual       = True,
    learning_rate    = 0.01,
    lradj            = 'type1',
    patience         = 3,
    save_path        = "./model2_ILI_E1_1E",
    attach_to_trend  = True,
    attach_to_seasonal = True
)
Path(base_cfg.save_path).mkdir(parents=True, exist_ok=True)

### Inference

In [ ]:
import torch, numpy as np
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import r2_score
from pathlib import Path

C        = 1                                
P        = base_cfg.pred_len                
seg_cnt  = 25
batch_sz = 32

results2_ILI_E1_test_1E = {}  

for seg in range(seg_cnt):
    print(f"\n―――― Inference  |  Segment {seg} ――――")

    
    pv = test_pred_segments_E1[:, seg, :, :]         
    ds = TensorDataset(test_sequences, test_labels, pv) 
    dl = DataLoader(ds, batch_size=batch_sz, shuffle=False)

    
    pred_ch, true_ch, metrics = [], [], {}

    
    for ch in range(C):
        ckpt = Path(base_cfg.save_path) / f"seg{seg}" / f"ch{ch}" / "checkpoint.pth"
        if not ckpt.exists():
            print(f"[Warn] seg{seg}-ch{ch} ckpt 없음 → skip");  continue

        
        model = Model(base_cfg).to(device)
        model.load_state_dict(torch.load(ckpt, map_location=device))
        model.eval()

        preds, trues = [], []
        with torch.no_grad():
            for xb, yb, pv_batch in dl:                 
                xb  = xb.float().to(device)
                yb  = yb.float()        
                pv_ = pv_batch.float().to(device)

                out = model(xb, pv_)                     
                preds.append(out[:, -P:, ch].cpu())
                trues.append(yb[:, -P:, ch])

        preds = torch.cat(preds, 0)     
        trues = torch.cat(trues, 0)
        pred_ch.append(preds)
        true_ch.append(trues)

        
        p, t = preds.reshape(-1).cpu().numpy(), trues.reshape(-1).cpu().numpy()
        mse  = np.mean((p - t)**2)
        mae  = np.mean(np.abs(p - t))
        metrics[ch] = (mse, mae)

        print(f"  seg{seg}-ch{ch}  MSE:{mse}  MAE:{mae}")

        del model; torch.cuda.empty_cache()

    
    pred_all = torch.stack(pred_ch,  dim=2)   
    true_all = torch.stack(true_ch,  dim=2)

    
    mse_avg = np.mean([m for m,_ in metrics.values()])
    mae_avg = np.mean([a for _,a in metrics.values()])
    print(f"▶ seg{seg}  mean  MSE:{mse_avg}  MAE:{mae_avg}")

    
    results2_ILI_E1_test_1E[seg] = {
        "pred2_E1_test_1E"    : pred_all,   
        "true2_E1_test_1E"    : true_all,   
        "metrics2_E1_test_1E" : metrics     
    }


seg10_ch0_mse = results2_ILI_E1_test_1E[10]["metrics2_E1_test_1E"][0][0]
print("\nseg10 ch0 MSE :", seg10_ch0_mse)



―――― Inference  |  Segment 0 ――――
  seg0-ch0  MSE:0.9163968563079834  MAE:0.7532691955566406
▶ seg0  mean  MSE:0.9163968563079834  MAE:0.7532691955566406

―――― Inference  |  Segment 1 ――――
  seg1-ch0  MSE:0.8625202178955078  MAE:0.672640860080719
▶ seg1  mean  MSE:0.8625202178955078  MAE:0.672640860080719

―――― Inference  |  Segment 2 ――――
  seg2-ch0  MSE:1.048213005065918  MAE:0.822629451751709
▶ seg2  mean  MSE:1.048213005065918  MAE:0.822629451751709

―――― Inference  |  Segment 3 ――――
  seg3-ch0  MSE:1.007404088973999  MAE:0.7880935072898865
▶ seg3  mean  MSE:1.007404088973999  MAE:0.7880935072898865

―――― Inference  |  Segment 4 ――――
  seg4-ch0  MSE:1.014391541481018  MAE:0.8087033033370972
▶ seg4  mean  MSE:1.014391541481018  MAE:0.8087033033370972

―――― Inference  |  Segment 5 ――――
  seg5-ch0  MSE:1.1148412227630615  MAE:0.8319649696350098
▶ seg5  mean  MSE:1.1148412227630615  MAE:0.8319649696350098

―――― Inference  |  Segment 6 ――――
  seg6-ch0  MSE:0.887762188911438  MAE:0.7128

# Rank Models (by 2nd validation performance)

In [49]:
results1_ILI_train = torch.load("./1st_results_ILI/train_1st_results.pt")
results1_ILI_val = torch.load("./1st_results_ILI/val_1st_results.pt")
results1_ILI_test = torch.load("./1st_results_ILI/test_1st_results.pt")

### ES->1E

In [ ]:

import numpy as np
from sklearn.metrics import mean_squared_error




TOP_N       = 25
SEG_CNT     = 25
CH_CNT      = 1
metric_idx  = 0          




mse_table = np.full((SEG_CNT, CH_CNT), np.inf)

for seg in range(SEG_CNT):
    if seg not in results2_ILI_ES_val_1E:
        continue
    for ch in range(CH_CNT):
        if ch in results2_ILI_ES_val_1E[seg]["metrics2_ES_val_1E"]:
            mse_table[seg, ch] = results2_ILI_ES_val_1E[seg]["metrics2_ES_val_1E"][ch][metric_idx]




print("\n▶ Train-set  Per-Channel  Top-N  MSE\n")
top_seg_idx_ES_val_1E = {}                       

for ch in range(CH_CNT):
    print(f"=== Channel {ch} ===")
    v_mse = results1_ILI_val[1]['mse_val_E1']
    print(f"Vanilla MSE : {v_mse:.6f}\n")

    
    pairs = [(seg, mse_table[seg, ch]) for seg in range(SEG_CNT)
             if np.isfinite(mse_table[seg, ch])]
    top   = sorted(pairs, key=lambda x: x[1])[:TOP_N]

    
    top_seg_idx_ES_val_1E[ch] = [seg for seg, _ in top]   

    
    for rank, (seg_id, mse_val) in enumerate(top, 1):
        delta = v_mse - mse_val
        sign  = "+" if delta > 0 else "-"
        print(f"  Top {rank:2d} | seg {seg_id:02d} | MSE {mse_val:.6f} "
              f"({sign}{abs(delta):.6f})")
    print()




print("Saved top segment indices per channel:")
for ch, seg_list in top_seg_idx_ES_val_1E.items():
    print(f"Channel {ch}: {seg_list}")



▶ Train-set  Per-Channel  Top-N  MSE

=== Channel 0 ===
Vanilla MSE : 0.166821

  Top  1 | seg 04 | MSE 0.077069 (+0.089753)
  Top  2 | seg 13 | MSE 0.105468 (+0.061353)
  Top  3 | seg 11 | MSE 0.109457 (+0.057364)
  Top  4 | seg 14 | MSE 0.112994 (+0.053827)
  Top  5 | seg 15 | MSE 0.113818 (+0.053003)
  Top  6 | seg 22 | MSE 0.116340 (+0.050481)
  Top  7 | seg 00 | MSE 0.118436 (+0.048385)
  Top  8 | seg 01 | MSE 0.126731 (+0.040091)
  Top  9 | seg 07 | MSE 0.128171 (+0.038650)
  Top 10 | seg 05 | MSE 0.128977 (+0.037844)
  Top 11 | seg 09 | MSE 0.130077 (+0.036744)
  Top 12 | seg 02 | MSE 0.133937 (+0.032884)
  Top 13 | seg 10 | MSE 0.139632 (+0.027189)
  Top 14 | seg 17 | MSE 0.143855 (+0.022967)
  Top 15 | seg 06 | MSE 0.143875 (+0.022946)
  Top 16 | seg 16 | MSE 0.144018 (+0.022804)
  Top 17 | seg 03 | MSE 0.145358 (+0.021463)
  Top 18 | seg 19 | MSE 0.146571 (+0.020250)
  Top 19 | seg 20 | MSE 0.150341 (+0.016480)
  Top 20 | seg 21 | MSE 0.152530 (+0.014291)
  Top 21 | seg 23 |

### 1E->1E

In [ ]:

import numpy as np
from sklearn.metrics import mean_squared_error




TOP_N       = 25
SEG_CNT     = 25
CH_CNT      = 1
metric_idx  = 0          




mse_table = np.full((SEG_CNT, CH_CNT), np.inf)

for seg in range(SEG_CNT):
    if seg not in results2_ILI_E1_val_1E:
        continue
    for ch in range(CH_CNT):
        if ch in results2_ILI_E1_val_1E[seg]["metrics2_E1_val_1E"]:
            mse_table[seg, ch] = results2_ILI_E1_val_1E[seg]["metrics2_E1_val_1E"][ch][metric_idx]




print("\n▶ Train-set  Per-Channel  Top-N  MSE\n")
top_seg_idx_E1_val_1E = {}                       

for ch in range(CH_CNT):
    print(f"=== Channel {ch} ===")
    v_mse = results1_ILI_val[1]['mse_val_E1']
    print(f"Vanilla MSE : {v_mse:.6f}\n")

    
    pairs = [(seg, mse_table[seg, ch]) for seg in range(SEG_CNT)
             if np.isfinite(mse_table[seg, ch])]
    top   = sorted(pairs, key=lambda x: x[1])[:TOP_N]

    
    top_seg_idx_E1_val_1E[ch] = [seg for seg, _ in top]   

    
    for rank, (seg_id, mse_val) in enumerate(top, 1):
        delta = v_mse - mse_val
        sign  = "+" if delta > 0 else "-"
        print(f"  Top {rank:2d} | seg {seg_id:02d} | MSE {mse_val:.6f} "
              f"({sign}{abs(delta):.6f})")
    print()




print("Saved top segment indices per channel:")
for ch, seg_list in top_seg_idx_E1_val_1E.items():
    print(f"Channel {ch}: {seg_list}")



▶ Train-set  Per-Channel  Top-N  MSE

=== Channel 0 ===
Vanilla MSE : 0.166821

  Top  1 | seg 00 | MSE 0.087757 (+0.079065)
  Top  2 | seg 18 | MSE 0.103841 (+0.062980)
  Top  3 | seg 15 | MSE 0.104231 (+0.062590)
  Top  4 | seg 23 | MSE 0.107200 (+0.059622)
  Top  5 | seg 22 | MSE 0.111953 (+0.054869)
  Top  6 | seg 16 | MSE 0.112173 (+0.054648)
  Top  7 | seg 03 | MSE 0.118512 (+0.048309)
  Top  8 | seg 04 | MSE 0.119836 (+0.046985)
  Top  9 | seg 05 | MSE 0.129199 (+0.037623)
  Top 10 | seg 09 | MSE 0.136024 (+0.030797)
  Top 11 | seg 19 | MSE 0.138379 (+0.028442)
  Top 12 | seg 10 | MSE 0.138705 (+0.028116)
  Top 13 | seg 01 | MSE 0.139012 (+0.027809)
  Top 14 | seg 12 | MSE 0.140445 (+0.026376)
  Top 15 | seg 02 | MSE 0.145943 (+0.020878)
  Top 16 | seg 20 | MSE 0.150219 (+0.016603)
  Top 17 | seg 08 | MSE 0.151818 (+0.015004)
  Top 18 | seg 11 | MSE 0.154972 (+0.011849)
  Top 19 | seg 14 | MSE 0.156611 (+0.010210)
  Top 20 | seg 17 | MSE 0.159734 (+0.007088)
  Top 21 | seg 21 |

# Ensemble

## ES->1E


In [ ]:
import torch, os, numpy as np
from torch.utils.data import DataLoader, TensorDataset

TOP_K    = 25
CH_CNT   = 1
batch_sz = 32

def infer_one(model, loader, ch):
    buf = []
    model.eval()
    with torch.no_grad():
        for xb, yb, pv in loader:
            xb, pv = xb.to(device), pv.to(device)
            buf.append(model(xb, pv)[:, :, ch].cpu())   
    return torch.cat(buf, 0)   


van_mse_list, ens_mse_list, delta_mse_list, delta_mse_pct_list = [], [], [], []
van_mae_list, ens_mae_list, delta_mae_list, delta_mae_pct_list = [], [], [], []




for ch in range(CH_CNT):
    print(f"\n=== Channel {ch} ===")

    
    
    
    v_mse = results1_ILI_test['ES']['mse_test_EES']
    v_mae = results1_ILI_test['ES']['mae_test_EES']

    print(f"[Vanilla]   MSE : {v_mse:.6f} | MAE : {v_mae:.6f}")

    van_mse_list.append(v_mse)
    van_mae_list.append(v_mae)

    preds_test_arr = []
    ok_segs = []

    
    for seg in top_seg_idx_ES_val_1E[ch][:TOP_K]:
        ckpt = f"./model2_ILI_EES_1E/seg{seg}/ch{ch}/checkpoint.pth"
        if not os.path.exists(ckpt):
            print(f"  [skip] ckpt missing : seg{seg}")
            continue

        pv_seg = test_pred_segments_ES[:, seg, :, :]
        dl     = DataLoader(TensorDataset(test_sequences, test_labels, pv_seg),
                            batch_size=batch_sz, shuffle=False)

        mdl = Model(base_cfg).to(device)
        mdl.load_state_dict(torch.load(ckpt, map_location=device))

        preds_test_arr.append(infer_one(mdl, dl, ch))   
        ok_segs.append(seg)

    if len(preds_test_arr) < 5:
        print("  Not enough valid models.")
        continue

    
    
    
    step = 5
    Ks = list(range(step, len(preds_test_arr)+1, step))

    var_list, rho_list = [], []

    for K in Ks:
        F = torch.stack(preds_test_arr[:K], dim=0)

        V = torch.var(F, dim=0).mean().item()
        var_list.append(V)

        flat = F.reshape(K, -1).numpy()
        corr = np.corrcoef(flat)
        R = np.mean(np.abs(corr[np.triu_indices(K, 1)]))
        rho_list.append(R)

    
    V_min, V_max = min(var_list), max(var_list)
    R_min, R_max = min(rho_list), max(rho_list)

    V_norm = [(v - V_min)/(V_max - V_min + 1e-12) for v in var_list]
    R_norm = [(r - R_min)/(R_max - R_min + 1e-12) for r in rho_list]

    alpha, beta = 1, 1
    scores = [alpha*v + beta*r for v, r in zip(V_norm, R_norm)]

    best_K = Ks[int(np.argmin(scores))]

    print("  — Var / |Corr| / Score —")
    for k, v, r, s in zip(Ks, var_list, rho_list, scores):
        tag = "<- pick" if k == best_K else ""
        print(f"   K={k:2d} | Var={v:.3e} | |ρ|={r:.4f} | S={s:.4f} {tag}")

    
    
    
    ens_pred = torch.mean(torch.stack(preds_test_arr[:best_K]), dim=0)
    y_true   = test_labels[:, -base_cfg.pred_len:, ch].cpu()

    mse_ens = torch.mean((ens_pred - y_true)**2).item()
    mae_ens = torch.mean(torch.abs(ens_pred - y_true)).item()

    ens_mse_list.append(mse_ens)
    ens_mae_list.append(mae_ens)

    
    delta_mse = v_mse - mse_ens
    delta_mae = v_mae - mae_ens

    
    delta_mse_pct = (delta_mse / v_mse) * 100
    delta_mae_pct = (delta_mae / v_mae) * 100

    delta_mse_list.append(delta_mse)
    delta_mae_list.append(delta_mae)

    delta_mse_pct_list.append(delta_mse_pct)
    delta_mae_pct_list.append(delta_mae_pct)

    print(f"[Var-Corr]  K={best_K}")
    print(f"  MSE = {mse_ens:.6f} | Δ = {delta_mse:+.6f} | Δ% = {delta_mse_pct:+.2f}%")
    print(f"  MAE = {mae_ens:.6f} | Δ = {delta_mae:+.6f} | Δ% = {delta_mae_pct:+.2f}%")




if ens_mse_list:
    print("\n──────── Summary (Var-Corr) ───────")
    print(f"Vanilla  mean MSE : {np.mean(van_mse_list):.6f}")
    print(f"Ensemble mean MSE : {np.mean(ens_mse_list):.6f}")
    print(f"Average Δ MSE     : {np.mean(delta_mse_list):+.6f}")
    print(f"Average Δ MSE (%) : {np.mean(delta_mse_pct_list):+.2f}%\n")

    print(f"Vanilla  mean MAE : {np.mean(van_mae_list):.6f}")
    print(f"Ensemble mean MAE : {np.mean(ens_mae_list):.6f}")
    print(f"Average Δ MAE     : {np.mean(delta_mae_list):+.6f}")
    print(f"Average Δ MAE (%) : {np.mean(delta_mae_pct_list):+.2f}%")



=== Channel 0 ===
[Vanilla]   MSE : 0.794277 | MAE : 0.740291
  — Var / |Corr| / Score —
   K= 5 | Var=2.510e-01 | |ρ|=0.8051 | S=1.0000 <- pick
   K=10 | Var=2.795e-01 | |ρ|=0.7929 | S=1.0439 
   K=15 | Var=2.831e-01 | |ρ|=0.7926 | S=1.0823 
   K=20 | Var=3.064e-01 | |ρ|=0.7796 | S=1.0357 
   K=25 | Var=3.270e-01 | |ρ|=0.7683 | S=1.0000 
[Var-Corr]  K=5
  MSE = 0.701944 | Δ = +0.092334 | Δ% = +11.62%
  MAE = 0.650516 | Δ = +0.089775 | Δ% = +12.13%

──────── Summary (Var-Corr) ───────
Vanilla  mean MSE : 0.794277
Ensemble mean MSE : 0.701944
Average Δ MSE     : +0.092334
Average Δ MSE (%) : +11.62%

Vanilla  mean MAE : 0.740291
Ensemble mean MAE : 0.650516
Average Δ MAE     : +0.089775
Average Δ MAE (%) : +12.13%


## 1E->1E


In [53]:
results1_ILI_test['ES']['mae_test_EES']

0.7402907013893127

In [ ]:
import torch, os, numpy as np
from torch.utils.data import DataLoader, TensorDataset

TOP_K    = 25
CH_CNT   = 1
batch_sz = 32

def infer_one(model, loader, ch):
    buf = []
    model.eval()
    with torch.no_grad():
        for xb, yb, pv in loader:
            xb, pv = xb.to(device), pv.to(device)
            buf.append(model(xb, pv)[:, :, ch].cpu())   
    return torch.cat(buf, 0)    



van_mse_list, ens_mse_list, delta_mse_list, delta_mse_pct_list = [], [], [], []
van_mae_list, ens_mae_list, delta_mae_list, delta_mae_pct_list = [], [], [], []





for ch in range(CH_CNT):

    print(f"\n=== Channel {ch} ===")

    
    v_mse = results1_ILI_test['ES']['mse_test_EES']
    v_mae = results1_ILI_test['ES']['mae_test_EES']

    print(f"[Vanilla]   MSE : {v_mse:.6f} | MAE : {v_mae:.6f}")

    van_mse_list.append(v_mse)
    van_mae_list.append(v_mae)

    preds_test_arr = []
    ok_segs = []

    
    
    
    for seg in top_seg_idx_E1_val_1E[ch][:TOP_K]:

        ckpt = f"./model2_ILI_E1_1E/seg{seg}/ch{ch}/checkpoint.pth"
        if not os.path.exists(ckpt):
            print(f"  [skip] missing checkpoint : seg{seg}")
            continue

        mdl = Model(base_cfg).to(device)
        mdl.load_state_dict(torch.load(ckpt, map_location=device))

        pv_seg = test_pred_segments_E1[:, seg, :, :]
        dl = DataLoader(
            TensorDataset(test_sequences, test_labels, pv_seg),
            batch_size=batch_sz, shuffle=False
        )

        preds_test_arr.append(infer_one(mdl, dl, ch))
        ok_segs.append(seg)

    if len(preds_test_arr) < 5:
        print("  Not enough valid models.")
        continue

    
    
    
    step = 5
    Ks = list(range(step, len(preds_test_arr)+1, step))

    var_list = []
    rho_list = []

    for K in Ks:
        F = torch.stack(preds_test_arr[:K], dim=0)   

        
        V = torch.var(F, dim=0).mean().item()
        var_list.append(V)

        
        flat = F.reshape(K, -1).numpy()
        corr = np.corrcoef(flat)
        R    = np.mean(np.abs(corr[np.triu_indices(K, 1)]))
        rho_list.append(R)

    
    
    
    V_min, V_max = min(var_list), max(var_list)
    R_min, R_max = min(rho_list), max(rho_list)

    V_norm = [(v - V_min) / (V_max - V_min + 1e-12) for v in var_list]
    R_norm = [(r - R_min) / (R_max - R_min + 1e-12) for r in rho_list]

    alpha, beta = 1, 1.1   

    scores = [alpha*v + beta*r for v, r in zip(V_norm, R_norm)]
    best_K = Ks[int(np.argmin(scores))]

    
    
    
    print("  — Var / |Corr| / Score —")
    for k, v, r, s in zip(Ks, var_list, rho_list, scores):
        tag = "<- pick" if k == best_K else ""
        print(f"   K={k:2d} | Var={v:.3e} | |ρ|={r:.4f} | S={s:.4f} {tag}")

    
    
    
    ens_pred = torch.mean(torch.stack(preds_test_arr[:best_K]), dim=0)  
    y_true   = test_labels[:, -base_cfg.pred_len:, ch].cpu()

    mse_ens = torch.mean((ens_pred - y_true)**2).item()
    mae_ens = torch.mean(torch.abs(ens_pred - y_true)).item()

    ens_mse_list.append(mse_ens)
    ens_mae_list.append(mae_ens)

    
    delta_mse = v_mse - mse_ens
    delta_mae = v_mae - mae_ens

    
    delta_mse_pct = (delta_mse / v_mse) * 100
    delta_mae_pct = (delta_mae / v_mae) * 100

    delta_mse_list.append(delta_mse)
    delta_mae_list.append(delta_mae)
    delta_mse_pct_list.append(delta_mse_pct)
    delta_mae_pct_list.append(delta_mae_pct)

    sign = "+" if delta_mse > 0 else "-"
    print(f"[Var-Corr]  K={best_K}")
    print(f"   MSE = {mse_ens:.6f} | Δ = {delta_mse:+.6f} | Δ% = {delta_mse_pct:+.2f}%")
    print(f"   MAE = {mae_ens:.6f} | Δ = {delta_mae:+.6f} | Δ% = {delta_mae_pct:+.2f}%")





if ens_mse_list:
    print("\n──────── Summary (Var-Corr) ───────")
    print(f"Vanilla  mean MSE  : {np.mean(van_mse_list):.6f}")
    print(f"Ensemble mean MSE  : {np.mean(ens_mse_list):.6f}")
    print(f"Average Δ MSE      : {np.mean(delta_mse_list):+.6f}")
    print(f"Average Δ MSE (%)  : {np.mean(delta_mse_pct_list):+.2f}%\n")

    print(f"Vanilla  mean MAE  : {np.mean(van_mae_list):.6f}")
    print(f"Ensemble mean MAE  : {np.mean(ens_mae_list):.6f}")
    print(f"Average Δ MAE      : {np.mean(delta_mae_list):+.6f}")
    print(f"Average Δ MAE (%)  : {np.mean(delta_mae_pct_list):+.2f}%")



=== Channel 0 ===
[Vanilla]   MSE : 0.794277 | MAE : 0.740291
  — Var / |Corr| / Score —
   K= 5 | Var=2.457e-01 | |ρ|=0.8040 | S=1.1000 
   K=10 | Var=2.876e-01 | |ρ|=0.7797 | S=0.9592 <- pick
   K=15 | Var=2.971e-01 | |ρ|=0.7775 | S=1.0135 
   K=20 | Var=3.137e-01 | |ρ|=0.7681 | S=0.9646 
   K=25 | Var=3.319e-01 | |ρ|=0.7613 | S=1.0000 
[Var-Corr]  K=10
   MSE = 0.758835 | Δ = +0.035442 | Δ% = +4.46%
   MAE = 0.706980 | Δ = +0.033311 | Δ% = +4.50%

──────── Summary (Var-Corr) ───────
Vanilla  mean MSE  : 0.794277
Ensemble mean MSE  : 0.758835
Average Δ MSE      : +0.035442
Average Δ MSE (%)  : +4.46%

Vanilla  mean MAE  : 0.740291
Ensemble mean MAE  : 0.706980
Average Δ MAE      : +0.033311
Average Δ MAE (%)  : +4.50%
